## Требуемые импорты

In [10]:
import os
import re
import json
import time
import whisper
import subprocess
import pandas as pd
import argostranslate.package
import argostranslate.translate

from gtts import gTTS
from pydub import AudioSegment
from moviepy.editor import *

## Транскрибирование текста и перевод на английский язык

In [4]:
def transcribe(file):
    subprocess.run(f"whisper {file} --language ru --task translate --model large-v3 --output_format all --device cuda --output_dir all --word_timestamps True", shell=True)

if __name__ == "__main__":
    video_folder = "videos"
    files = os.listdir(video_folder)
    for file in files:
        if file.endswith(".mp4"):
            file_path = os.path.join(video_folder, file)
            transcribe(file_path)

/home/isibgatullina/miniconda3/envs/inna_dir/lib/python3.10/site-packages/whisper/transcribe.py:151: UserWarning: Word-level timestamps on translations may not be reliable.
  warnings.warn("Word-level timestamps on translations may not be reliable.")


[00:00.000 --> 00:01.580]  Friends, good morning to everyone!
[00:01.580 --> 00:06.840]  The second day of the digital breakthrough in the Artificial Intelligence season begins with the Q&A session.
[00:06.880 --> 00:11.760]  I think you have a lot of questions, so get ready and don't hesitate to ask them.
[00:12.140 --> 00:14.460]  This can help you a lot in solving the case.
[00:14.660 --> 00:16.740]  Well, I will be moderating one of the sessions.
[00:16.840 --> 00:18.940]  Now I choose which one is the most difficult.
[00:19.260 --> 00:21.480]  Rosatom, Central Bank or OOO Solution?
[00:21.640 --> 00:22.240]  Please tell me.


In [54]:
TRANSCRIBE_LANG = 'en'

## Коды языков для последующей подстановки

In [2]:
code_lang = pd.read_csv('dataset.csv')
code_lang.head(40)

,target_language,video_name,lang_code
0,Немецкий,0.mp4,de
1,Английский,1.mp4,en
2,Французский,2.mp4,fr
3,Итальянский,3.mp4,it
4,Испанский,4.mp4,es
5,Английский,5.mp4,en
6,Японский,6.mp4,ja
7,Английский,7.mp4,en
8,Итальянский,8.mp4,it
9,Китайский,9.mp4,zh


## Перевод текстов в требуемый язык

In [58]:
def translate_text(file_path, from_code, to_code):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    argostranslate.package.update_package_index()
    available_packages = argostranslate.package.get_available_packages()
    package_to_install = next(
        filter(
            lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
        )
    )
    argostranslate.package.install_from_path(package_to_install.download())

    translatedText = argostranslate.translate.translate(text, from_code, to_code)
    return translatedText

     
srt_folder = './all' 
translated_folder = './all' 

for filename in os.listdir(result_folder):
    if filename.endswith('.txt'):
        file_number = filename.split('.')[0]

        file_row = code_lang[code_lang['video_name'] == f"{file_number}.mp4"]
        
        if not file_row.empty:
            target_lang = file_row['lang_code'].values[0]
            if target_lang == 'en':
                continue
            file_path = os.path.join(srt_folder, filename)
            translated_text = translate_text(file_path, TRANSCRIBE_LANG, target_lang)
            translated_text = re.sub(r'www.\S+', '', translated_text)
            
            translated_file_path = os.path.join(translated_folder, f"{file_number}.txt")
            with open(translated_file_path, 'w', encoding='utf-8') as file:
                file.write(translated_text)
            
            print(target_lang)
            print("-------------------------------")
            print(translated_text)

ja
-------------------------------
1
00:00:00,000 --> 00:00:11,560
そして、一番下にあるコンテンツの一部はスクロールし続けますので、ページの上部にある動画の留め具と呼ばれています。

2
00:00:11,820 --> 00:00:19,380
ここは既にあったと思いましたが、今はやったので、素晴らしい仕事をしました。 とても便利です。

3
00:00:19,740 --> 00:00:28,160
はい。しかし、これは便利な使い方と機能全体を照らすために必要な基本的な機能の改善です。

3
00:00:28,160 --> 00:00:43,220
また、私たちのユーザーにとって、ブロガーにとって、特別なミニプレイヤーは、ブロガーに添付されたビデオが再生されているチャンネルのページに表示されます。

5月5日
00:00:43,360 --> 00:00:56,700
ブロガーが彼の主要なビデオの一部を強調することができるので、彼は昇進できるか、彼は最も成功したことを考慮する必要があります。

6月6日
00:00:56,820 --> 00:00:57,960
最近の投稿

7月7日
00:00:58,160 --> 00:01:01,180
最近のことは、誰もがこのために異なる理由を持つことができます。

8月8日
00:01:02,000 --> 00:01:15,880
そして、加入者、またはむしろ、加入者ではない聴衆、彼らは、このビデオを見てきた、超クール、スーパーファッショナブルな、この決定をより速くすることができるようになります。

9月9日
00:01:16,280 --> 00:01:28,140
そして、それに応じて、ブロガーに既に登録されている聴衆は、例えば、このブロガーが今持っているものを見るための非常に基本的な欲求を持つことができます。

10月10日
00:01:28,140 --> 00:01:47,780
ビデオでキャッチアップを取得しないために、いくつかのプレイリストを見ることではなく、この同じブロガーは、加入者に別のビデオを投稿することができます。これにより、ビューを増やすのに役立ちます。そして、ユーザーはすぐにトップビデオを見る機会を得ることができます。



## Генерация звуковых дорожек дубляжа

In [ ]:
translated_folder = './all' 

for filename in os.listdir(translated_folder):
    if filename.endswith('.txt'):
        file_number = filename.split('.')[0]

        file_row = code_lang[code_lang['video_name'] == f"{file_number}.mp4"]
        
        if not file_row.empty:
            target_lang = file_row['lang_code'].values[0]
            
        path_file = os.path.join(translated_folder, filename)
        
        with open(path_file, 'r', encoding='utf-8') as file:
            transleated_text = file.read()
        tts = gTTS(transleated_text, lang=target_lang)
        tts.save(f'translate_audio/{file_number}.mp3')
        time.sleep(30)

In [25]:
!mv all/*.json json/

## TTS и наложение звуковой дорожки дубляжа на оригинальную звуковую дорожку с уменьшением громкости последней

In [ ]:
videos_folder = "videos"
src_audio_folder = "src_audio"
translate_audio_folder = "translate_audio/mp3"
json_folder = "json"

for filename in os.listdir(videos_folder):
    if filename.endswith(".mp4"):
        video_path = os.path.join(videos_folder, filename)
        video_name = filename.split(".")[0]

        json_data = os.path.join(json_folder, f"{video_name}.json")

        with open(json_data, encoding="utf-8") as fh:
            parsed_data = json.load(fh)

        segments = parsed_data["segments"]
        first_word = segments[0]["words"][0]["word"]
        start_first_word = segments[0]["words"][0]["start"]

        audio = VideoFileClip(video_path).audio
        audio.write_audiofile(f"src_audio/{video_name}.mp3")

        audio_clip1 = AudioFileClip(os.path.join(src_audio_folder, f"{video_name}.mp3"))
        audio_clip2 = AudioFileClip(os.path.join(translate_audio_folder, f"{video_name}.mp3"))

        audio_clip1 = audio_clip1.volumex(0.06)

        final_audio = CompositeAudioClip([audio_clip1, audio_clip2.set_start(start_first_word)])
        final_audio.write_audiofile(f"{video_name}_mix.mp3", fps=44100)

        video = VideoFileClip(video_path)
        new_audio = AudioFileClip(f"{video_name}_mix.mp3")

        video = video.set_audio(new_audio)
        video.write_videofile(f"res_vi/{video_name}.mp4", codec="libx264", audio_codec="aac")

MoviePy - Writing audio in src_audio/40.mp3


MoviePy - Done.
MoviePy - Writing audio in 40_mix.mp3


MoviePy - Done.
Moviepy - Building video res_vi/40.mp4.
MoviePy - Writing audio in 40TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video res_vi/40.mp4



Moviepy - Done !
Moviepy - video ready res_vi/40.mp4
MoviePy - Writing audio in src_audio/66.mp3


...

Moviepy - Done !
Moviepy - video ready res_vi/13.mp4
MoviePy - Writing audio in src_audio/83.mp3


MoviePy - Done.
MoviePy - Writing audio in 83_mix.mp3


MoviePy - Done.
Moviepy - Building video res_vi/83.mp4.
MoviePy - Writing audio in 83TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video res_vi/83.mp4



t:  27%|███████████████████████████████████████▌                                                                                                           | 6057/22501 [00:33<01:14, 221.51it/s, now=None]